In [1]:
!pip install datasets
!pip install accelerate

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 k

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, generation, DataCollatorWithPadding, TrainingArguments, Trainer, EarlyStoppingCallback
import torch
import tensorflow as tf
from datasets import Dataset

In [2]:
df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [3]:
checkpoint_name= 'juan-glez29/marIA-ideologiamul-none-label'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def concatenar_hasta_limite(tweets, max_longitud=None):
    segmentos = []
    segmento_actual = ""

    for tweet in tweets:
        # Tokenizar el segmento actual junto con el nuevo tweet
        # y contar el número total de tokens
        token_count = len(tokenizer(segmento_actual + tweet + " ", add_special_tokens=False)['input_ids'])

        # Revisar la longitud del próximo segmento potencial en términos de tokens
        if token_count <= max_longitud:
            segmento_actual += tweet
        else:
            # Guardar el segmento actual y comenzar uno nuevo
            segmentos.append(segmento_actual.strip())
            segmento_actual = tweet

    # No olvidar agregar el último segmento
    if segmento_actual:
        segmentos.append(segmento_actual.strip())

    return segmentos

# Asumiendo que df_test_full es tu DataFrame y ya está cargado
df_segmentado = pd.DataFrame()

# Usar una longitud máxima de tokens (por ejemplo, 512 para BERT)
max_tokens = 511

for (label, ideology), group in df_test.groupby(['label', 'ideology_multiclass']):
    segmentos = concatenar_hasta_limite(group['tweet'], max_tokens)
    segmento_data = [{'label': label, 'ideology_multiclass': ideology, 'tweet': segmento} for segmento in segmentos]
    df_segmentado = pd.concat([df_segmentado, pd.DataFrame(segmento_data)], ignore_index=True)



In [19]:
# Mostrar las primeras filas del DataFrame resultante
df_segmentado.head(10)

,label,ideology_multiclass,tweet
0,015f536ad54d267bef6d9a08019f735f,moderate_right,"Ayer, en Guadalajara: feministas arrojaban áci..."
1,015f536ad54d267bef6d9a08019f735f,moderate_right,"¡¡¡BRAVO!!!. @user : ""en vez de recurrir a ges..."
2,015f536ad54d267bef6d9a08019f735f,moderate_right,Alberto Fernández dice que los jugadores no fu...
3,015f536ad54d267bef6d9a08019f735f,moderate_right,🔴 La delegada de Ceuta culpa a Marlaska e Inte...
4,015f536ad54d267bef6d9a08019f735f,moderate_right,“Dialogar”: aún con 62% diciendo NO. La centro...
5,015f536ad54d267bef6d9a08019f735f,moderate_right,Parece que los progres también son biológicame...
6,015f536ad54d267bef6d9a08019f735f,moderate_right,@user La Iglesia tampoco es aceptar que 2 + 2 ...
7,015f536ad54d267bef6d9a08019f735f,moderate_right,"@user Curiosa lógica, en la que vandalizar edi..."
8,015f536ad54d267bef6d9a08019f735f,moderate_right,La encuesta más acertada sobre los hispanos de...
9,0281b081d05d7eeeca8be8cd6fbda012,moderate_left,"Pues ya he comido, un besi a [POLITICAL_PARTY]..."


In [20]:
df_test= df_segmentado.copy()

In [21]:
df_test.drop(['label'], axis=1, inplace=True)

##  Clasificador

In [22]:
label_encoder = LabelEncoder()

df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])


In [23]:
def tokenize_function(example):
    return tokenizer(example['tweet'])

dataset_test = Dataset.from_pandas(df_test)

In [24]:
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)
tokenized_dataset_test= tokenized_dataset_test.rename_column("ideology_multiclass", "labels")
test_dataset= tokenized_dataset_test.remove_columns(["tweet"])
test_dataset

Map:   0%|          | 0/3625 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3625
})

In [25]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

labels = test_dataset["labels"]

macro_f1 = f1_score(labels, y_pred, average='macro')# o weighted si estan desbalceadas las clases

print(f"F1-Score:\t{macro_f1}")

F1-Score:	0.5386195341110657
